In [1]:
import EEGAnalysis as ea

import numpy as np
import pandas as pd
import h5py
from pprint import pprint
import os, re, json, shutil, random
from hashlib import sha256
from tqdm import tqdm

import matplotlib.pyplot as plt
from IPython import display

In [2]:
_data_dir = './Data'  ## set data folder
ea_manager = ea.DataManager(_data_dir)  ## create data manager

patient_id = 'fansulong'
patient = ea_manager.get_patient(patient_id)  ## load patient

In [3]:
roi = (-2, 5)
tspec = np.linspace(roi[0], roi[1], (roi[1] - roi[0]) * 2000)
frange = np.logspace(np.log10(1), np.log10(150), 30)

In [4]:
for chidx in tqdm(range(len(patient._sgch_config['chidx']))):
    entry = patient.load_isplit(chidx, '20190110-1')

    _epoch = ea.create_1d_epoch_bymarker(entry['20190110-1']['value'], patient.get_marker('20190110-1')[:20], (-2,5), 2000)

    _dwt_result = ea.decomposition.dwt.dwt(data=_epoch, frange=frange, fs=2000, reflection=True)
    _pwr = ea.decomposition.dwt_power(dwtresult=_dwt_result, fs=2000, zscore=True)

    plt.figure(figsize=(8,6))
    plt.contourf(tspec, frange, _pwr, 80)
    plt.clim((-3 * np.std(_pwr), 3 * np.std(_pwr)))
#     plt.colorbar()
#     plt.show()
    plt.tight_layout()
    plt.savefig('export/heatmap/ch_%03d.png'%chidx, bbox_inches='tight')
    plt.close()


100%|██████████| 73/73 [06:46<00:00,  5.86s/it]


In [9]:
loadedf = ea.io.loadedf
detect_cross_pnt = ea.decomposition.detect_cross_pnt

class _DEV(ea.datamanager.Patient):
    def update_DC_marker(self, overwrite=False):
        _date_pattern = r'(\d{8})-(\d)'
        for item in self._raw_config.values():
            if item['name'] in list(self._marker.file) and not overwrite:
                continue
            elif item['name'] in list(self._marker.file) and overwrite:
                print('overwrite the markers of %s'%(item['name']))
            else:
                pass
            
            _edf = loadedf(item['file'], 'parse marker')
            
            try:
                _grating_marker_ch = np.where([True if item == 'POL DC10' else False for item in _edf.channelLabels])[0][0]
                _clapping_marker_ch = np.where([True if item == 'POL DC09' else False for item in _edf.channelLabels])[0][0]
                print("file %s: DC10: %d"%(item['name'], _grating_marker_ch))
            except IndexError:
                print('file %s has no DC channels'%item['name'])
                continue
                
            _date, _paradigm = re.findall(_date_pattern, item['name'])[0]
            try:
                _df = pd.read_csv(os.path.join(self._marker_dir, '%s-%s.csv'%(_date, self.id)))
            except FileNotFoundError:
                print('file %s has no corresponding behavior.csv'%item['file'])
            
            _paradigm_name = [item for item in _df[_df['mode'] == 'grating']['paradigm'].values if re.match(r'1-\d{1,2}', item)]
            
            _grating_trace = _edf.data[_grating_marker_ch] * _edf.physical_unit[_grating_marker_ch] / 1e6  # unit as Volt
#             _clapping_trace = _edf.data[_clapping_marker_ch] * _edf.physical_unit[_clapping_marker_ch] / 1e6  # unit as Volt
            
            _grating_time = np.array(detect_cross_pnt(_grating_trace, 3, gap=_edf.fs)) / _edf.fs
#             _clapping_time = np.array(detect_cross_pnt(_clapping_trace, 3, gap=_edf.fs)) / _edf.fs
            
            _grating_markers = [{'file':item['name'], 'paradigm':'', 'marker':_i, 'mbias':'0','note':''} for _i in _grating_time]
#             _clapping_markers = [{'file':item['name'], 'paradigm':'', 'marker':_i, 'mbias':'0','note':''} for _i in _clapping_time]
            
            self._marker = self._marker.append(_grating_markers)
#             self._behavior = self._behavior.append(_clapping_markers)
            self._update_marker()
            
    def get_marker(self, name, paradigm=None):
        if paradigm == None:
            _temp = self._marker.marker[self._marker.file == name]+\
                self._marker.mbias[self._marker.file == name]
            return _temp.values
        else:
            _temp = self._marker.marker[(self._marker.file == name)&(self._marker.paradigm == name)]+\
                self._marker.mbias[(self._marker.file == name)&(self._marker.paradigm == name)]
            return _temp.values

patient = _DEV('./Data', 'fansulong')


In [4]:
_date, _paradigm = '20190110', '2'

In [5]:
_df = pd.read_csv(os.path.join(patient._marker_dir, '%s-%s.csv'%(_date, patient.id)))

In [6]:
patient._marker

,file,paradigm,marker,mbias,note
0,20190110-2,NaN,25.506,0,NaN
1,20190110-2,NaN,30.527,0,NaN
2,20190110-2,NaN,35.563,0,NaN
3,20190110-2,NaN,40.600,0,NaN
4,20190110-2,NaN,45.618,0,NaN
5,20190110-2,NaN,50.638,0,NaN
6,20190110-2,NaN,55.675,0,NaN
7,20190110-2,NaN,60.693,0,NaN
8,20190110-2,NaN,65.712,0,NaN
9,20190110-2,NaN,70.731,0,NaN


In [7]:
[item for item in _df[_df['mode'] == 'grating'].paradigm.values if re.match(r'1-\d{1,2}', item)]

['1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-5',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10',
 '1-10']

In [8]:
patient.update_DC_marker()

FileNotFoundError: File b'./Data/fansulong/EEG/Marker/20190109-fansulong.csv' does not exist